# Setup

##### Set CWD

In [1]:
cd ..

d:\Documents\A_DIGIPEN\PersonalSVN\Fall22SVN\CSP400\MRB_II


##### Imports

In [5]:
import pandas as pd

from MySQLdb import _mysql

import googleapiclient.discovery

# Data

In [20]:
def build_client(path="./secret_keys/"):
    """Creates and returns the YouTube Data API client needed for requests."""
    print("building client...")

    api_key_path = path + "api_key.txt"

    # API information
    api_service_name = "youtube"
    api_version = "v3"

    # Read API key
    with open(api_key_path, "r") as api_file:
        api_key = api_file.read()

    # Create the client with api_key
    youtube = googleapiclient.discovery.build(
        api_service_name, api_version, developerKey=api_key
    )
    print("client built")

    return youtube

In [21]:
youtube = build_client()


building client...
client built


# Database

In [23]:
with open("mysql/mysql_key.txt", "r") as file:
    key = file.read()

db = _mysql.connect("localhost", "eric", key, "test")